In [1]:
#imports
#pip install numpy
import numpy as np
#pip install pandas
import pandas as pd
#pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow import keras
#standard library
import os

In [2]:
#this function allows to read the csv files from the logs of the attentive cursor dataset
#path_logs : path to the logs of the dataset with the files containing the list of inputs for each user
#path_csv : path to the groundtruth.csv containing the attention score of the each user
#dfs : array containing, for each user, its attention score and the count of its inputs
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [3]:
#loading the data
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [4]:
#keeping the attention score and the amount of each input for each user
cleandf = []
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

In [5]:
#removing 'useless input types from the data'
tab = np.array(cleandf, dtype='object')
removed = ['mouseup','focus','load','beforeunload','unload','touchend'] #arbitrary list of input types to remove
for row in tab :
    for input in removed : 
        if input in row[1].keys() :
            row[1].pop(input)

#searching the data for every other input types and storing them in alphabetical order
all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)
all_inputs.sort()
print(all_inputs)

['blur', 'click', 'contextmenu', 'copy', 'keydown', 'keyup', 'mousedown', 'mousemove', 'mouseover', 'resize', 'scroll', 'select', 'touchmove', 'touchstart']


In [6]:
#normaliations and the removal of extremes data are kept commented because they did not affect the accuracy
#of the model in any significant way but are kept in the code for future devs to know what has been tried on
#this model

#normalisation v1
"""
for input in all_inputs :
    tmp = []
    for row in tab :
        if input in row[1].keys() :
            tmp.append(row[1].get(input))
    mean = np.mean(tmp)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/mean

print(tab)
"""
#normalisation v3
"""
for input in all_inputs :
    max = 0
    for row in tab :
        if input in row[1].keys() :
            if row[1].get(input) > max :
                max = row[1].get(input)
    for row in tab :
        if input in row[1].keys() :
            row[1][input] = row[1][input]/max
"""

'\nfor input in all_inputs :\n    max = 0\n    for row in tab :\n        if input in row[1].keys() :\n            if row[1].get(input) > max :\n                max = row[1].get(input)\n    for row in tab :\n        if input in row[1].keys() :\n            row[1][input] = row[1][input]/max\n'

In [7]:
tmp_all_x = [] #stock a python array version of the data shaped as the entry of the model
tmp_all_y = [] #stock a python array version of the data shaped as the results of the model (for training purpose)

#the entry data for the model for each user in an array with a length of 14 (the number of input types) filled
# with number of inputs for each input type in the order of the list of input types 

for row in tab :
    tmp_all_x.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_all_x[len(tmp_all_x)-1].append(row[1].get(input))
        else :
            tmp_all_x[len(tmp_all_x)-1].append(0)
    tmp_all_y.append(row[0])

all_x = np.array(tmp_all_x) #numpy array version of the shaped data
all_y = np.array(tmp_all_y)
print(all_x)

[[3 2 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [2 1 0 ... 0 0 0]
 [3 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


In [8]:
#normalisation v2
"""
for i in range(len(all_inputs)) :
    print(i)
    tmp = []
    for row in all_x :
        tmp.append(row[i])
    mean = np.mean(tmp)
    for row in all_x :
        row[i] = row[i]/mean
    
print(all_x)
"""

#enlevage des données extrèmes (faire une normalisation avant)
"""
i = 0
bad_rows = []
for row in range(len(all_x)) :
    for element in all_x[row] :
        if element > 5 :
            i +=1
            bad_rows.append(row)
            continue

all_x = np.delete(all_x,bad_rows, axis=0)
print(all_x)
"""

'\ni = 0\nbad_rows = []\nfor row in range(len(all_x)) :\n    for element in all_x[row] :\n        if element > 5 :\n            i +=1\n            bad_rows.append(row)\n            continue\n\nall_x = np.delete(all_x,bad_rows, axis=0)\nprint(all_x)\n'

In [9]:
#this is a classifier model
inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(1, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

#list of inputs (uncoment to see)
print('all used input types : ')
print(all_inputs)
#all_inputs correspond to all the inputs type minus
print('all removed inputs : ')
print(removed)

all used input types : 
['blur', 'click', 'contextmenu', 'copy', 'keydown', 'keyup', 'mousedown', 'mousemove', 'mouseover', 'resize', 'scroll', 'select', 'touchmove', 'touchstart']
all removed inputs : 
['mouseup', 'focus', 'load', 'beforeunload', 'unload', 'touchend']


In [10]:
print(all_x[1])

[0 1 0 0 0 0 1 9 3 0 7 0 0 0]


In [11]:
history = model.fit(all_x, all_y, epochs=20, validation_split=0.1)

Epoch 1/20
82/82 [==============================] - 1s 3ms/step - loss: 1.7886 - accuracy: 0.3640 - val_loss: 1.7824 - val_accuracy: 0.4227
Epoch 2/20
82/82 [==============================] - 0s 2ms/step - loss: 1.7801 - accuracy: 0.3678 - val_loss: 1.7732 - val_accuracy: 0.4227
Epoch 3/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7718 - accuracy: 0.3678 - val_loss: 1.7642 - val_accuracy: 0.4227
Epoch 4/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7635 - accuracy: 0.3682 - val_loss: 1.7553 - val_accuracy: 0.4227
Epoch 5/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7553 - accuracy: 0.3678 - val_loss: 1.7466 - val_accuracy: 0.4227
Epoch 6/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7474 - accuracy: 0.3686 - val_loss: 1.7380 - val_accuracy: 0.4227
Epoch 7/20
82/82 [==============================] - 0s 1ms/step - loss: 1.7398 - accuracy: 0.3686 - val_loss: 1.7300 - val_accuracy: 0.4227
Epoch 8/20
82/82 [==

In [12]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, './model')

In [13]:
num_array = np.array([ 0, 17, 0, 0, 0, 0, 21, 324, 28, 0, 0, 0, 0, 0 ])
prediction = model.predict(num_array.reshape(1, -1))
print(prediction)

1/1 [==============================] - 0s 71ms/step
[[0.02905064 0.03532463 0.06507348 0.03190719 0.5299927  0.30865136]]


In [14]:
#attempt at a linear reggression model
data = all_x

data_normalizer = keras.layers.Normalization(input_shape=[14,], axis=None)
data_normalizer.adapt(data)

persuasiveness_model = tf.keras.Sequential([
    data_normalizer,
    keras.layers.Dense(units=1)
])

persuasiveness_model.summary()

persuasiveness_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mae'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 14)               3         
 n)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 18
Trainable params: 15
Non-trainable params: 3
_________________________________________________________________


In [15]:
history = persuasiveness_model.fit(all_x, all_y, epochs=100, validation_split=0.2)

Epoch 1/100
73/73 [==============================] - 0s 3ms/step - loss: 1.3435 - mae: 1.3435 - val_loss: 0.9934 - val_mae: 0.9934
Epoch 2/100
73/73 [==============================] - 0s 1ms/step - loss: 1.0649 - mae: 1.0649 - val_loss: 0.9518 - val_mae: 0.9518
Epoch 3/100
73/73 [==============================] - 0s 2ms/step - loss: 1.0714 - mae: 1.0714 - val_loss: 1.0295 - val_mae: 1.0295
Epoch 4/100
73/73 [==============================] - 0s 2ms/step - loss: 1.0708 - mae: 1.0708 - val_loss: 0.9583 - val_mae: 0.9583
Epoch 5/100
73/73 [==============================] - 0s 1ms/step - loss: 1.0657 - mae: 1.0657 - val_loss: 0.9965 - val_mae: 0.9965
Epoch 6/100
73/73 [==============================] - 0s 1ms/step - loss: 1.0876 - mae: 1.0876 - val_loss: 0.9943 - val_mae: 0.9943
Epoch 7/100
73/73 [==============================] - 0s 1ms/step - loss: 1.0620 - mae: 1.0620 - val_loss: 0.9838 - val_mae: 0.9838
Epoch 8/100
73/73 [==============================] - 0s 1ms/step - loss: 1.0757 - m

In [16]:
persuasiveness_model.predict(data[:10])

1/1 [==============================] - 0s 35ms/step


array([[4.0737286],
       [4.0480556],
       [4.0317826],
       [4.0442758],
       [4.051831 ],
       [4.040819 ],
       [4.039178 ],
       [4.0420923],
       [4.0452876],
       [4.0548716]], dtype=float32)